In [1]:
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

In [2]:
class Colors:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    RESET = '\033[0m'

# 데이터 크롤링

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=3)

# 반복 종료 조건
loop = True

# 네이버 지도에서 검색할 위치
search_location = "잠실야구장 주변맛집"

# 네이버 지도 검색 페이지로 이동
driver.get(f"https://map.naver.com/v5/search/{search_location}")

# 맛집 이름과 평점 크롤링
restaurants = []
while(True):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)

        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절

        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break

        last_height = new_height


    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')
    
    #현재가게 찾기(프린트 화면으로)
    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(2)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름        
        rating = 0.0 # 평점        
        address = '' # 가게 주소
        phone_num = '' # 전화번호
        switch_left()

        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()
        
        sleep(2)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        #식당이름
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        #가게정보
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')
        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text
        # 리뷰
        review = title.find_elements(By.XPATH,'.//div[2]/span')
        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
         # 가게 전화번호
        phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text
        
        # 요소를 찾기 위한 XPath
        xpaths = ['//*[@id="ibu_1"]', '//*[@id="ibu_2"]','//*[@id="ibu_3"]','//*[@id="ibu_4"]']
        # 요소를 찾기 위한 XPath
#        xpaths = ['//*[@id="ibu_1"]', '//*[@id="ibu_2"]','//*[@id="cp0_2"]','//*[@id="app-root"]/div/div/div/div[1]/div/div[1]/div/a/span','//*[@id="app-root"]/div/div/div/div[1]/div/a/img']  
        # 요소를 찾을 변수
        img_src = None
        #  XPath를 순차적으로 시도
        for xpath in xpaths:
            elements = driver.find_elements(By.XPATH, xpath)
            
            if elements:
                first_element = elements[0]  # 첫 번째 요소 선택
                
                if first_element.tag_name == 'img':  # 첫 번째 요소가 이미지인지 확인
                    img_src = first_element.get_attribute('src')
                    print("First image source found:", img_src)
                    break  # 이미지를 찾았으므로 반복 종료
                
                elif first_element.tag_name == 'video':  # 첫 번째 요소가 동영상인 경우
                    print("First element is a video. Searching for the third image...")
                    
                    # 세 번째 이미지 요소가 있는지 확인
                    img_elements = [el for el in elements if el.tag_name == 'img']
                    
                    if len(img_elements) >= 2:  # 두 번째 이미지가 있는지 확인
                        img_src = img_elements[1].get_attribute('src')  # 세 번째 이미지의 'src' 가져오기
                        print("Third image source found:", img_src)
                        break  # 세 번째 이미지를 찾았으므로 반복 종료
                    elif img_elements: #이미지가 하나만 있는경우
                        img_src = first_element.get_attribute('src')
                        print("Less than three images found in the current XPath.")
                        break
                    else:
                        print("No images found in the current XPath.")
            
            print(f"No valid image found at XPath: {xpath}, trying next XPath...")
            
        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1
        
        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text
 
        
        # 데이터 저장
        restaurants.append({
            "store_name": store_name,
            "rating":rating,
            "category":category,
            "address": address,
            "phone_num": phone_num,
            "img":img_src
        })
    switch_left()
    break
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False

# 데이터 프레임으로 저장
rf = pd.DataFrame(restaurants)     

현재 1 페이지 / 총 68개의 가게를 찾았습니다.

1. 스시유 삼성본점
2. 마케집 잠실점
3. 쭈꾸미도사 잠실새내점
4. 우곱집 잠실본점
5. 찬란한아구 잠실새내점
6. 잠실장어와 한우
7. 샤이바나 코엑스점
8. 원조부안집 잠실새내점
9. 동어동락 삼성본점
10. 강남면옥 신천점
11. 삼성골목집 삼성본점
12. 잠잠소 잠실야구장과잠실새내역 회식좋은소고기한우맛집
13. 목구멍 잠실새내점
14. 해주냉면 본점
15. 중앙해장
16. 우리소 삼성점
17. 돈순장 잠실새내 본점
18. 수다주막
19. 요리하는남자
20. 압구정화로구이 잠실본점
21. 김인복의광평 삼성본점
22. 담산 잠실새내점
23. 제주애월당
24. 도몽 잠실새내 본점
25. 옥된장 잠실새내점
26. 공인인증소 잠실새내점
27. 오하이요 잠실새내점
28. 철길부산집 잠실새내점
29. 바이킹스워프 코엑스점
30. 부엉이산장 잠실새내점
31. 깃뽀 잠실새내점
32. 국고집
33. 보노보노 삼성점
34. 래빗캐슬
35. 루아
36. 다이닝갈비 잠실새내점
37. 그릴1492 선릉점
38. 바토스 파르나스몰점
39. 크라이치즈버거 삼성역점
40. 이나세
41. 대홍집 잠실새내본점
42. 구주 잠실새내점
43. 반포식스 포스코사거리점
44. 대성정육식당 잠실새내역본점
45. 무월식탁 파르나스점
46. 더플레이스 스타필드 코엑스점
47. 하우스서울
48. 고쿠텐 잠실새내점
49. 천미미 삼성점
50. 잠실백정
51. 팀호완 삼성점
52. 늘푸른목장 잠실본점
53. 용용선생 선릉점
54. 오레노카츠 선릉점
55. 허머스키친
56. 딤딤섬 파르나스몰점
57. 신선계 잠실새내본점
58. 파크 하얏트 서울 코너스톤
59. 상구네돼지구이 잠실새내점
60. 솔솥 포스코센터점
61. 파오파오
62. 영동가든
63. 이남장 삼성점
64. 도명골 청국장
65. 이치고
66. 민규 삼성점
67. 옥소반 잠실새내점
68. 무한소
--------------------------------------------------
F

In [13]:
rf

,store_name,rating,category,address,phone_num,img
0,스시유 삼성본점,0.0,일식당,서울 강남구 영동대로 407 코니빌딩,010-2781-7871,https://search.pstatic.net/common/?autoRotate=...
1,마케집 잠실점,0.0,이자카야,"서울 송파구 올림픽로10길 19 1, 2층",0507-1483-4008,https://search.pstatic.net/common/?autoRotate=...
2,쭈꾸미도사 잠실새내점,0.0,주꾸미요리,서울 송파구 백제고분로7길 30 1층,0507-1305-0524,https://search.pstatic.net/common/?autoRotate=...
3,우곱집 잠실본점,0.0,"곱창,막창,양",서울 송파구 백제고분로7길 16-20 1층,0507-1338-9798,https://search.pstatic.net/common/?autoRotate=...
4,찬란한아구 잠실새내점,0.0,"아귀찜,해물찜",서울 송파구 백제고분로9길 36 1층,0507-1393-3453,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...,...
63,도명골 청국장,별점 4.63,"찌개,전골",서울 강남구 봉은사로114길 38 101호 도명골 청국장,0507-1457-7015,https://search.pstatic.net/common/?autoRotate=...
64,이치고,0.0,이자카야,서울 송파구 백제고분로11길 6-7 1층,02-412-0481,https://search.pstatic.net/common/?autoRotate=...
65,민규 삼성점,0.0,"육류,고기요리",서울 강남구 봉은사로84길 9 1층 민규 삼성점,0507-1369-0356,https://search.pstatic.net/common/?autoRotate=...
66,옥소반 잠실새내점,0.0,샤브샤브,서울 송파구 백제고분로9길 10 1층,0507-1352-5486,https://search.pstatic.net/common/?autoRotate=...


In [14]:
# 데이터프레임을 CSV 파일로 저장
csv_file_path = '잠실야구장사진맛집.csv'
rf.to_csv(csv_file_path, index=False)  # index=False로 인덱스를 제외하고 저장

In [17]:
rf = pd.read_csv('잠실야구장사진맛집.csv')

In [18]:
rf

,store_name,rating,category,address,phone_num,img
0,스시유 삼성본점,0.0,일식당,서울 강남구 영동대로 407 코니빌딩,010-2781-7871,https://search.pstatic.net/common/?autoRotate=...
1,마케집 잠실점,0.0,이자카야,"서울 송파구 올림픽로10길 19 1, 2층",0507-1483-4008,https://search.pstatic.net/common/?autoRotate=...
2,쭈꾸미도사 잠실새내점,0.0,주꾸미요리,서울 송파구 백제고분로7길 30 1층,0507-1305-0524,https://search.pstatic.net/common/?autoRotate=...
3,우곱집 잠실본점,0.0,"곱창,막창,양",서울 송파구 백제고분로7길 16-20 1층,0507-1338-9798,https://search.pstatic.net/common/?autoRotate=...
4,찬란한아구 잠실새내점,0.0,"아귀찜,해물찜",서울 송파구 백제고분로9길 36 1층,0507-1393-3453,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...,...
63,도명골 청국장,별점 4.63,"찌개,전골",서울 강남구 봉은사로114길 38 101호 도명골 청국장,0507-1457-7015,https://search.pstatic.net/common/?autoRotate=...
64,이치고,0.0,이자카야,서울 송파구 백제고분로11길 6-7 1층,02-412-0481,https://search.pstatic.net/common/?autoRotate=...
65,민규 삼성점,0.0,"육류,고기요리",서울 강남구 봉은사로84길 9 1층 민규 삼성점,0507-1369-0356,https://search.pstatic.net/common/?autoRotate=...
66,옥소반 잠실새내점,0.0,샤브샤브,서울 송파구 백제고분로9길 10 1층,0507-1352-5486,https://search.pstatic.net/common/?autoRotate=...


In [19]:
raddress = rf['address']
raddress

0                서울 강남구 영동대로 407 코니빌딩
1             서울 송파구 올림픽로10길 19 1, 2층
2                서울 송파구 백제고분로7길 30 1층
3             서울 송파구 백제고분로7길 16-20 1층
4                서울 송파구 백제고분로9길 36 1층
                   ...               
63    서울 강남구 봉은사로114길 38 101호 도명골 청국장
64             서울 송파구 백제고분로11길 6-7 1층
65         서울 강남구 봉은사로84길 9 1층 민규 삼성점
66               서울 송파구 백제고분로9길 10 1층
67                  서울 송파구 백제고분로9길 25
Name: address, Length: 68, dtype: object

In [318]:
#쪼개기 활용화
# traddress = rf[['address']].copy()
# traddress.head()

,address
0,대구 수성구 명덕로 485
1,대구 수성구 동대구로42길 7
2,대구 수성구 들안로 51-1 1층
3,대구 수성구 동대구로59길 14-2 101호
4,대구 수성구 들안로 44 숨쉬는순두부


In [20]:
import re
# "층", "호" 및 주소의 끝에 있는 추가 정보 제거하고 숫자 번지수만 남기는 함수 정의
# 주소에서 숫자 번지수와 그 이전의 주소 부분만 유지하는 함수 정의
def clean_address(address):
    # 주소에서 숫자 번지수와 그 이전의 주소 부분만 유지하고 그 이후의 정보를 제거
    match = re.search(r'^.*?\d+(?:-\d+)?(?:\s|$)', address)
    if match:
        return match.group().strip()  # 추출된 주소 부분만 반환
    return address.strip()

In [320]:
#traddress['address'] = traddress['address'].apply(clean_address)

In [21]:
raddress = rf['address'].apply(clean_address)

In [22]:
raddress

0          서울 강남구 영동대로 407
1        서울 송파구 올림픽로10길 19
2        서울 송파구 백제고분로7길 30
3     서울 송파구 백제고분로7길 16-20
4        서울 송파구 백제고분로9길 36
              ...         
63      서울 강남구 봉은사로114길 38
64     서울 송파구 백제고분로11길 6-7
65        서울 강남구 봉은사로84길 9
66       서울 송파구 백제고분로9길 10
67       서울 송파구 백제고분로9길 25
Name: address, Length: 68, dtype: object

# 위도 경도 값으로 바꿔주기

In [23]:
####### 도로명주소 위도 경도 값으로 바꿔주기 ########
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(raddress):
    try:
        geo = geo_local.geocode(raddress)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]
#####주소를 위,경도 값으로 변환하기 #####
latitude = []
longitude =[]

for i in raddress:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

#####Dataframe만들기######
new = pd.DataFrame({'store_name': rf['store_name'],'rating':rf['rating'],'category':rf['category'],'phone_num':rf['phone_num'],'img':rf['img'],'address':raddress,'latitude':latitude,'longitude':longitude})
new.head()

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,스시유 삼성본점,0.0,일식당,010-2781-7871,https://search.pstatic.net/common/?autoRotate=...,서울 강남구 영동대로 407,37.505604,127.064392
1,마케집 잠실점,0.0,이자카야,0507-1483-4008,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 올림픽로10길 19,37.508431,127.081648
2,쭈꾸미도사 잠실새내점,0.0,주꾸미요리,0507-1305-0524,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로7길 30,37.509675,127.082310
3,우곱집 잠실본점,0.0,"곱창,막창,양",0507-1338-9798,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로7길 16-20,37.509675,127.082310
4,찬란한아구 잠실새내점,0.0,"아귀찜,해물찜",0507-1393-3453,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로9길 36,37.509107,127.079414


In [24]:
new

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,스시유 삼성본점,0.0,일식당,010-2781-7871,https://search.pstatic.net/common/?autoRotate=...,서울 강남구 영동대로 407,37.505604,127.064392
1,마케집 잠실점,0.0,이자카야,0507-1483-4008,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 올림픽로10길 19,37.508431,127.081648
2,쭈꾸미도사 잠실새내점,0.0,주꾸미요리,0507-1305-0524,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로7길 30,37.509675,127.082310
3,우곱집 잠실본점,0.0,"곱창,막창,양",0507-1338-9798,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로7길 16-20,37.509675,127.082310
4,찬란한아구 잠실새내점,0.0,"아귀찜,해물찜",0507-1393-3453,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로9길 36,37.509107,127.079414
...,...,...,...,...,...,...,...,...
63,도명골 청국장,별점 4.63,"찌개,전골",0507-1457-7015,https://search.pstatic.net/common/?autoRotate=...,서울 강남구 봉은사로114길 38,37.513028,127.064941
64,이치고,0.0,이자카야,02-412-0481,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로11길 6-7,37.508110,127.081729
65,민규 삼성점,0.0,"육류,고기요리",0507-1369-0356,https://search.pstatic.net/common/?autoRotate=...,서울 강남구 봉은사로84길 9,37.512216,127.055922
66,옥소반 잠실새내점,0.0,샤브샤브,0507-1352-5486,https://search.pstatic.net/common/?autoRotate=...,서울 송파구 백제고분로9길 10,37.509107,127.079414


In [25]:
#0점을 미등록으로 등록하기
new['rating'] = new['rating'].replace('0','미등록')

In [392]:
#new.rename(columns={'위도':'latitude','경도':'longitude'}, inplace=True)

In [27]:
# 데이터프레임을 CSV 파일로 저장
csv_file_path = '잠실역맛집 위도 이미지 경도버전.csv'
new.to_csv(csv_file_path, index=False)  # index=False로 인덱스를 제외하고 저장

# DB 적재

In [28]:
import cx_Oracle

In [29]:
# 라이브러리 임포트
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData

In [30]:
# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

# 커서 생성
cursor = connection.cursor()

In [40]:
# 테이블 생성 쿼리
create_table_query = '''
create table irestaurants(
    store_name varchar(50),
    rating varchar(30),
    category varchar(30),
    address varchar(100),
    phone_num varchar(50),
    latitude number(9, 6),
    longitude number(9, 6)
)
'''
# 쿼리 실행
cursor.execute(create_table_query)

#변경 사항 커밋
connection.commit()

#커서와 연결 종료
cursor.close()
connection.close()

#성공 알람
print("Table created successfully!!")

Table created successfully!!


In [59]:
# #테이블 삭제 쿼리(필요시 사용할것)
# drop_restaurants = "DROP TABLE restaurants"
# #쿼리 실행
# cursor.execute(drop_restaurants)

# #변경 사항 커밋
# connection.commit()

# #커서와 연결 종료
# cursor.close()
# connection.close()

# #성공 알람
# print("Table drop successfully!!")

Table drop successfully!!


# 식당 정보를 DB에 적재

In [31]:
from sqlalchemy import create_engine
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')
new.to_sql(
    'irestaurants', 
    con=engine, 
    if_exists='append', 
    index=False, 
    dtype={
        'store_name': VARCHAR2(50),
        'rating': VARCHAR2(30),
        'category': VARCHAR2(30),
        'address' :VARCHAR2(100),
        'phone_num': VARCHAR2(50),
        'img':VARCHAR2(500),
        'latitude': NUMBER(precision=9, scale=6), #위도
        'longitude': NUMBER(precision=9, scale=6)  #경도
    }
)
print("Table insert successfully")

Table insert successfully


In [60]:
#db에서 가져옴
query = "select * from irestaurants"
drdf = pd.read_sql(query, con=engine)

In [61]:
drdf.head()

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,아트플렉스 수성못,미등록,"카페,디저트",0507-1441-8278,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 용학로 92-4,35.823710,128.642880
1,르배 본점,별점 4.44,베이커리,053-384-2464,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 화랑로8길 11-11,35.871276,128.634643
2,거송갈비찜 범어점,별점 4.46,"육류,고기요리",0507-1482-3346,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 명덕로 485,35.854355,128.617134
3,수페부엌 범어점,별점 4.65,패밀리레스토랑,0507-1331-7106,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로42길 7,35.849402,128.625688
4,패대기 두산들안길점,별점 4.85,"육류,고기요리",0507-1312-2588,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 들안로 51-1,35.864875,128.618419


In [73]:
import folium

# 수성구의 위도와 경도
latitude = 35.8576054
longitude = 128.634312675189

#지도를 생성(위도와 경도 기준)
# 지도를 생성 (위도와 경도 기준)
map_suseong = folium.Map(location=[latitude, longitude], zoom_start=15)

# 마커 추가
for store_name, rating, adress, phone_num, cetegory, image,lat, lng in zip(drdf['store_name'],drdf['rating'],drdf['address'],drdf['phone_num'],drdf['category'],
    drdf['img'],drdf['latitude'],drdf['longitude']):
    popup_html = f"""
    <img src="{image}" width="200px" />
    <b>가게이름:</b> {store_name}<br>
    <b>평점:</b>{rating}<br>
    <b>카테고리:</b> {category}<br>
    <b>주소:</b> {adress}<br>
    <b>전화번호:</b>{phone_num}<br>
    
    """
    
    folium.Marker(
        location=[lat, lng],
#        popup=store_name,
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(icon="cloud"),
        ).add_to(map_suseong)

# 지도를 HTML 파일로 저장
map_suseong.save("suseong_restaurants.html")
map_suseong